In [78]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def getRequest(url):
    try:
        return BeautifulSoup(requests.get(url).content, "html.parser")        
    except Exception as e:
        print("The URL failed to return")
        print(getattr(e, 'message', repr(e)))
       

url = "https://www.racingaustralia.horse"
main_page_url = "https://www.racingaustralia.horse/home.aspx"

# results = [i['href'] for i in soup.select('[href*=Calendar_Results]')]

results = [i['href'] for i in getRequest(main_page_url).select('[href*=Calendar_Results]')]
    
stateResults = []

for item in results:
    state = url + item
    stateResults.append("%s" % state)
    
# print(stateResults)
# # 
#TODO Review option to use /InteractiveForm/TrackCondition.aspx?State=NSW
# interim = stateResults[0]
gardens = []

#TODO determine track names to retrieve URLS (endswith)
for item in stateResults:
    for state_url in getRequest(item).findAll('a'):
        if state_url.attrs['href'].endswith('Gardens'):
            gardens.append(f"{url}{state_url['href']}")

# removes list duplicates
url_list = list(dict.fromkeys(gardens))
# Gets Meeting Dates.
for item in url_list:
    try:
        print(getRequest(item).find('span').text)
    except:
        print(f"maybe a bad url {getRequest(item)}")

keys = []
values = []

def find_by_label(soup, label):
    return soup.find("b", text=re.compile(label)).next_sibling

output = []

def getStats(urls, len_url_list):
    while len_url_list > 0:
        for url in urls:
            stats = getRequest(url).find('div', {'class': 'race-venue-bottom'})
            for data in stats.findAll('b'):
                keys.append(data.text)
            for key in keys:
                values.append(find_by_label(stats, key).strip()) 
        len_url_list =  len_url_list - 1
        yield keys, values

for k, v in getStats(url_list, len(url_list)):
    output.append(dict(zip(k, v)))
    
# print(output)

# dataframe = pd.read_csv('C:\Users\sayth\Documents\Data')
df = pd.DataFrame(output)
df.to_csv(r'C:\Users\sayth\Documents\Data\track_data.csv', encoding='utf-8', index=False)

Saturday, 26 June 2021
Saturday, 19 June 2021
Saturday, 05 June 2021
